Group 9 Project 
Riley Fritz, Sofanit Habte, Robert Boulka
Loading transformed Data from 2019 reported fires in the city of Minneapolis

In [ ]:
#importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import password

In [ ]:
#making connection to SQL database
rds_connection_string = "postgres:{password}@localhost:5432/fire_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
conn = engine.connect()

Extracting and Transforming the fires reported data

In [ ]:
#Load csv
file = 'Resources/Fires_Reported_2019_YTD.csv'
df1 = pd.read_csv(file)

In [ ]:
#organizing data 
df2 = df1.loc[:,["inci_no", "alm_date", "number", "street", "st_type", "st_suffix", "latitude", "longitude", "inci_type"]]


In [ ]:
#transforming and joining street address columns
df2['street_address'] = df2['street'] + " " + df2['st_type'] + " " + df2['st_suffix']


In [ ]:
#dropping the "19-" from incident number column
df2['inci_no'] = df2['inci_no'].str[3:]

In [ ]:
#dropping timestamp from datetime column
df2['alm_date'] = pd.to_datetime(df2['alm_date'])
df2['date'] = df2['alm_date'].dt.date

In [ ]:
#renaming columns to match database table
df3 = df2.rename(columns = {'inci_no': 'incident_number', 'number': 'street_number', 'inci_type': 'incident_type', 'latitude': 'latitude', 'longitude': 'longitude',})
df3.head()

In [ ]:
#reorganizing columns to match database table
fires_reported_df = df3[['incident_number', 'date', 'incident_type', 'street_number', 'street_address', 'latitude', 'longitude']]
fires_reported_df.head()

In [ ]:
#loading dataframe into database table
fires_reported_df.to_sql(name='fires_reported', con=engine, if_exists='append', index=False)